In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from functools import reduce

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
 # Calculate the number of unique schools in the DataFrame
school_count = len(school_data_complete["school_name"].unique())

#Calculate the total number of students in the DataFrame
student_count =len(school_data_complete["student_name"])

#Calculate the total budget
total_budget="${:,.2f}".format(sum(school_data["budget"]))

#Calcuate the average math score
average_math=format(school_data_complete["math_score"].mean(), '.2f')

#Calculate the average reading score
average_reading=format(school_data_complete["reading_score"].mean(), '.2f')

#Calculate the percentage of students with a passing math score (70 or greater)
passing_math_subset=school_data_complete[school_data_complete["math_score"] >= 70]
passing_math=format(len(passing_math_subset["math_score"])/student_count*100, '.2f')

#Calculate the percentage of students with a passing reading score (70 or greater)
passing_reading_subset=school_data_complete[school_data_complete["reading_score"] >= 70]
passing_reading=format(len(passing_reading_subset["reading_score"])/student_count*100, '.2f')

#Calculate the percentage of students who passed math and reading (% overall passing)
overall_passing_subset=school_data_complete[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)]
overall_passing=format(len(overall_passing_subset["reading_score"])/student_count*100, '.2f')

In [3]:
 # Place all of the data found into a summary DataFrame
district_summary = pd.DataFrame({"Total Schools": [school_count],
                              "Total Students": [student_count],
                              "Total Budget": [total_budget],
                              "Average Math Score": [average_math],
                                "Average Reading Score": [average_reading],
                                 "% Passing Math": [passing_math],
                                 "% Passing Reading": [passing_reading],
                                    "% Overall Passing": [overall_passing]})
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",78.99,81.88,74.98,85.81,65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [4]:
#Create new data frame with additional and removed columns from school_data_complete
school_summary = school_data_complete.drop(['Student ID', 'gender', 'grade', 'School ID', 'size'], axis=1)

In [5]:
#Add Per Student Budget Column
school_summary['Per Student Budget']= school_summary['budget']

#Add % Passing Math Score column
school_summary['Passing Math']=np.where(school_summary["math_score"] >= 70, 1, 0)

#Add % Passing Reading Score column
school_summary['Passing Reading']=np.where(school_summary["reading_score"] >= 70, 1, 0)

#Add % Passing Overall column
school_summary['Passing Overall']=school_summary["reading_score"].between(70, 100, inclusive=1) & school_summary['math_score'].between(70, 100, inclusive=1)

In [109]:
#Create groupby
schools_group = school_summary.groupby(['school_name'])

schoolSummary1=school_summary.groupby(['school_name'],as_index=False).agg({'type':pd.Series.mode})

#Aggregate apply function for columns in school summary
def f(x):
    d = {}
#    d['School Type']=x['type'].mode()
    d['Total Students']=x['student_name'].count()
    d['Total School Budget']=x['budget'].median()
    d['Per Student Budget']=x['Per Student Budget'].median()/len(x)
    d['Average Math Score']=x['math_score'].mean()
    d['Average Reading Score']=x['reading_score'].mean()
    d['% Passing Math'] = x['Passing Math'].sum()/len(x)*100
    d['% Passing Reading'] = x['Passing Reading'].sum()/len(x)*100
    d['% Passing Overall']=x['Passing Overall'].sum()/len(x)*100
    return pd.Series(d, index=['Total Students', 'Total School Budget', 'Per Student Budget', 
                               'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading',
                               '% Passing Overall'])

schoolSummary2=school_summary.groupby('school_name').apply(f)

In [110]:
#Combine summary tables
schoolSummaryCombined = pd.merge(schoolSummary1, schoolSummary2, on='school_name', how='left')

#Reorder columns
schoolSummaryCombined_reordered = schoolSummaryCombined[["school_name", "type", "Total Students", "Total School Budget",
                                                         "Per Student Budget", "Average Math Score", "Average Reading Score",
                                                         "% Passing Math", "% Passing Reading", "% Passing Overall"
                                                  ]]
#Rename remaining columns
schoolSummaryCombined_renamed = schoolSummaryCombined_reordered.rename(columns={'school_name':'School Name', 'type' : 'Type'})

#Set index as School Name
schoolSummaryCombined_renamed=schoolSummaryCombined_renamed.set_index('School Name')
schoolSummaryCombined_renamed

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Name,,,,,,,,,
Bailey High School,District,4976.0,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858.0,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949.0,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739.0,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468.0,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635.0,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427.0,248087.0,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917.0,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761.0,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [112]:
schoolSummaryCombined_topFive = schoolSummaryCombined_renamed.sort_values(["% Passing Overall"], ascending=False)
schoolSummaryCombined_topFive.head()

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Name,,,,,,,,,
Cabrera High School,Charter,1858.0,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635.0,1043130.0,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468.0,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283.0,1319574.0,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962.0,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [113]:
schoolSummaryCombined_bottomFive = schoolSummaryCombined_renamed.sort_values(["% Passing Overall"], ascending=True)
schoolSummaryCombined_bottomFive.head()

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Name,,,,,,,,,
Rodriguez High School,District,3999.0,2547363.0,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949.0,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917.0,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635.0,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761.0,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [45]:
#Calculate the average math score for 9th grade
ninthMathSubset=student_data[school_data_complete["grade"] == '9th']
ninthMathTotal=pd.DataFrame({'School Name': ninthMathSubset['school_name'], '9th': ninthMathSubset['math_score']})

#Calculate the average math score for 10th grade
tenthMathSubset=student_data[school_data_complete["grade"] == '10th']
tenthMathTotal=pd.DataFrame({'School Name': tenthMathSubset['school_name'], '10th': tenthMathSubset['math_score']})

#Calculate the average math score for 11th grade
eleventhMathSubset=student_data[school_data_complete["grade"] == '11th']
eleventhMathTotal=pd.DataFrame({'School Name': eleventhMathSubset['school_name'], '11th': eleventhMathSubset['math_score']})

#Calculate the average math score for 12th grade
twelthMathSubset=student_data[school_data_complete["grade"] == '12th']
twelthMathTotal=pd.DataFrame({'School Name': twelthMathSubset['school_name'], '12th': twelthMathSubset['math_score']})

In [51]:
#Create groupby for 9th grade
ninthMathGroup = ninthMathTotal.groupby(["School Name"])
ninthMathAverage = ninthMathGroup.mean()

#Create groupby for 10th grade
tenthMathGroup = tenthMathTotal.groupby(["School Name"])
tenthMathAverage = tenthMathGroup.mean()

#Create groupby for 11th grade
eleventhMathGroup = eleventhMathTotal.groupby(["School Name"])
eleventhMathAverage = eleventhMathGroup.mean()

#Create groupby for 12th grade
twelthMathGroup = twelthMathTotal.groupby(["School Name"])
twelthMathAverage = twelthMathGroup.mean()

In [57]:
#Compile list of data frames for grades
frames = [ninthMathAverage, tenthMathAverage, eleventhMathAverage, twelthMathAverage]

#Merge data frames
mathByGrade = reduce(lambda  left,right: pd.merge(left,right,on=['School Name'],
                                            how='outer'), frames)

#Print summary
mathByGrade.round(decimals=2)

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [55]:
#Calculate the average reading score for 9th grade
ninthReadingSubset=student_data[school_data_complete["grade"] == '9th']
ninthReadingTotal=pd.DataFrame({'School Name': ninthReadingSubset['school_name'], '9th': ninthReadingSubset['reading_score']})

#Calculate the average reading score for 10th grade
tenthReadingSubset=student_data[school_data_complete["grade"] == '10th']
tenthReadingTotal=pd.DataFrame({'School Name': tenthReadingSubset['school_name'], '10th': tenthReadingSubset['reading_score']})

#Calculate the average reading score for 11th grade
eleventhReadingSubset=student_data[school_data_complete["grade"] == '11th']
eleventhReadingTotal=pd.DataFrame({'School Name': eleventhReadingSubset['school_name'], '11th': eleventhReadingSubset['reading_score']})

#Calculate the average math score for 12th grade
twelthReadingSubset=student_data[school_data_complete["grade"] == '12th']
twelthReadingTotal=pd.DataFrame({'School Name': twelthReadingSubset['school_name'], '12th': twelthReadingSubset['reading_score']})

#Create groupby for 9th grade
ninthReadingGroup = ninthReadingTotal.groupby(["School Name"])
ninthReadingAverage = ninthReadingGroup.mean()

#Create groupby for 10th grade
tenthReadingGroup = tenthReadingTotal.groupby(["School Name"])
tenthReadingAverage = tenthReadingGroup.mean()

#Create groupby for 11th grade
eleventhReadingGroup = eleventhReadingTotal.groupby(["School Name"])
eleventhReadingAverage = eleventhReadingGroup.mean()

#Create groupby for 12th grade
twelthReadingGroup = twelthReadingTotal.groupby(["School Name"])
twelthReadingAverage = twelthReadingGroup.mean()

#Compile list of data frames for grades
reading_frames = [ninthReadingAverage, tenthReadingAverage, eleventhReadingAverage, twelthReadingAverage]

#Merge data frames
readingByGrade = reduce(lambda  left,right: pd.merge(left,right,on=['School Name'],
                                            how='outer'), reading_frames)

#Print summary
readingByGrade.round(decimals=2)

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [135]:
#Create new table
schoolSummaryBins = schoolSummaryCombined_renamed.drop(['Total School Budget'], axis=1)

#Create bins for spending
spendingBins = [0, 584, 629, 644, 675]

#Create labels for these bins
spending_group_labels = ["<$584", "$585-629", "$630-644", "$645-675"]

In [142]:
#Slice the data and place it into bins
pd.cut(schoolSummaryBins["Per Student Budget"], spendingBins, labels=spending_group_labels).head()

School Name
Bailey High School      $585-629
Cabrera High School        <$584
Figueroa High School    $630-644
Ford High School        $630-644
Griffin High School     $585-629
Name: Per Student Budget, dtype: category
Categories (4, object): [<$584 < $585-629 < $630-644 < $645-675]

In [152]:
 # Place the data series into a new column inside of the DataFrame
schoolSummaryBins["Spending Ranges (Per Student)"] = pd.cut(schoolSummaryBins["Per Student Budget"], bins, labels=group_labels)

# Create a GroupBy object based upon "Spending Ranges (Per Student)"
summarySpending_group = schoolSummaryBins.groupby("Spending Ranges (Per Student)")

In [153]:
summarySpending_group[["Average Math Score", "Average Reading Score", "% Passing Math",
                      "% Passing Reading", "% Passing Overall"]].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Spending Ranges (Per Student),,,,,
<$584,83.455399,83.933814,93.460096,96.610877,90.369459
$585-629,81.899826,83.155286,87.133538,92.718205,81.418596
$630-644,78.518855,81.624473,73.484209,84.391793,62.857656
$645-675,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

* Perform the same operations as above, based on school size.

In [139]:
#Create bins for spending
sizeBins = [0, 1000, 2000, 5000]

#Create labels for these bins
size_group_labels = ["Small (<1000)", "Medium (1000-2000)", "Large(2000-5000)"]

In [140]:
#Slice the data and place it into bins
pd.cut(schoolSummaryBins["Total Students"], sizeBins, labels=size_group_labels).head()

School Name
Bailey High School        Large(2000-5000)
Cabrera High School     Medium (1000-2000)
Figueroa High School      Large(2000-5000)
Ford High School          Large(2000-5000)
Griffin High School     Medium (1000-2000)
Name: Total Students, dtype: category
Categories (3, object): [Small (<1000) < Medium (1000-2000) < Large(2000-5000)]

In [159]:
 # Place the data series into a new column inside of the DataFrame
schoolSummaryBins["School Size"] = pd.cut(schoolSummaryBins["Total Students"], sizeBins, labels=size_group_labels)

# Create a GroupBy object based upon "Spending Ranges (Per Student)"
summarySize_group = schoolSummaryBins.groupby("School Size")

In [158]:
summarySize_group[["Average Math Score", "Average Reading Score", "% Passing Math",
                      "% Passing Reading", "% Passing Overall"]].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large(2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [163]:
#Group School Summary by Type
schoolTypeGroup = schoolSummaryCombined_renamed.groupby(['Type'])

#Show average scores & passing columns
schoolTypeGroup[["Average Math Score", "Average Reading Score", "% Passing Math",
                      "% Passing Reading", "% Passing Overall"]].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
